# Introduction to SageMaker JumpStart - Time Series Forecasting with Chronos

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

---

In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy a SageMaker JumpStart time series forecasting model and invoke the endpoint.

## Setup
First, upgrade to the latest sagemaker SDK to ensure all available models are deployable.

In [1]:
%pip install sagemaker --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


Select the desired model to deploy. The provided dropdown filters all time series forecasting models available in SageMaker JumpStart.

In [2]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

dropdown = Dropdown(
    options=list_jumpstart_models(filter="task == forecasting"),
    value="autogluon-forecasting-chronos-t5-small",
    description="Select a JumpStart time series forecasting model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(dropdown)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/shchuro/.config/sagemaker/config.yaml


Dropdown(description='Select a JumpStart time series forecasting model:', index=2, layout=Layout(width='max-co…

In [3]:
model_id = dropdown.value
model_version = "*"

## Deploy model

Create a `JumpStartModel` object, which initializes default model configurations conditioned on the selected instance type. JumpStart already sets a default instance type, but you can deploy the model on other instance types by passing `instance_type` to the `JumpStartModel` class.

In [6]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)

You can now deploy the model using SageMaker JumpStart. The deployment might take a few minutes. 

In [7]:
predictor = model.deploy()

----------!

## Invoke the endpoint

This section demonstrates how to invoke the endpoint using example payloads that are retrieved programmatically from the `JumpStartModel` object. You can replace these example payloads with your own payloads.

In [8]:
from pprint import pformat


def nested_round(data, decimals=3):
    """Round numbers, including nested dicts and list."""
    if isinstance(data, float):
        return round(data, decimals)
    elif isinstance(data, list):
        return [nested_round(item, decimals) for item in data]
    elif isinstance(data, dict):
        return {key: nested_round(value, decimals) for key, value in data.items()}
    else:
        return data


def pretty_format(data):
    return pformat(nested_round(data), width=150, sort_dicts=False)

In [15]:
for payload in model.retrieve_all_examples():
    response = predictor.predict(payload.body)
    print("Input:\n", pretty_format(payload.body), end="\n\n")
    print("Output:\n", pretty_format(response))
    print("\n===============\n")

Input:
 {'inputs': [{'target': [0.0, 4.0, 5.0, 1.5, -3.0, -5.0, -3.0, 1.5, 5.0, 4.0, 0.0, -4.0, -5.0, -1.5, 3.0, 5.0, 3.0, -1.5, -5.0, -4.0]}],
 'parameters': {'prediction_length': 10}}

Output:
 {'predictions': [{'mean': [-0.488, 3.101, 3.086, 0.436, -2.867, -3.924, -2.258, 0.686, 2.456, 2.089],
                  '0.1': [-4.331, 1.351, -0.04, -3.467, -5.713, -5.051, -5.056, -3.247, -1.907, -1.898],
                  '0.5': [0.0, 3.507, 4.012, 0.0, -3.003, -4.903, -3.015, 1.501, 3.003, 3.003],
                  '0.9': [1.652, 4.997, 4.997, 4.11, 0.0, -2.215, 1.602, 4.997, 4.997, 4.997]}]}


Input:
 {'inputs': [{'target': [1.0, 2.0, 3.0, 2.0, 0.5, 2.0, 3.0, 2.0, 1.0], 'item_id': 'product_A', 'start': '2024-01-01T01:00:00'},
            {'target': [5.4, 3.0, 3.0, 2.0, 1.5, 2.0, -1.0], 'item_id': 'product_B', 'start': '2024-02-02T03:00:00'}],
 'parameters': {'prediction_length': 5, 'freq': '1h', 'quantile_levels': [0.05, 0.5, 0.95], 'num_samples': 30, 'batch_size': 2}}

Output:
 {'predict

The payload for Chronos models must be structured as follows.
* **inputs** (required): List with at most 64 time series that need to be forecasted. Each time series is represented by a dictionary with the following keys:
    * **target** (required): List of observed numeric time series values. 
        - It is recommended that each time series contains at least 30 observations.
        - If any time series contains fewer than 5 observations, an error will be raised.
    * **item_id**: String that uniquely identifies each time series. 
        - If provided, the ID must be unique for each time series.
        - If provided, then the endpoint response will also include the **item_id** field for each forecast.
    * **start**: Timestamp of the first time series observation in ISO format (`YYYY-MM-DD` or `YYYY-MM-DDThh:mm:ss`). 
        - If **start** field is provided, then **freq** must also be provided as part of **parameters**.
        - If provided, then the endpoint response will also include the **start** field indicating the first timestamp of each forecast.
* **parameters**: Optional parameters to configure the model.
    * **prediction_length**: Integer corresponding to the number of future time series values that need to be predicted. 
        - Recommended to keep prediction_length <= 64 since larger values will result in inaccurate quantile forecasts. Values above 1000 will raise an error.
    * **quantile_levels**: List of floats in range (0, 1) specifying which quantiles should should be included in the probabilistic forecast. Defaults to `[0.1, 0.5, 0.9]`. 
    * **freq**: Frequency of the time series observations in [pandas-compatible format](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases). For example, `1h` for hourly data or `2W` for bi-weekly data. 
        - If **freq** is provided, then **start** must also be provided for each time series in **inputs**.
    * **num_samples**: Number of sample trajectories generated by the Chronos model during inference. Larger values may improve accuracy but increase memory consumption and slow down inference. Defaults to `20`.
    * **batch_size**: Number of time series processed in parallel by the model. Larger values speed up inference but may lead to out of memory errors.

All keys not marked with (required) are optional.

The endpoint response contains the probabilistic (quantile) forecast for each time series included in the request.

## Clean up the endpoint
Don't forget to clean up resources when finished to avoid unnecessary charges.

In [16]:
predictor.delete_predictor()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/generative_ai|sm-jumpstart_time_series_forecasting.ipynb)
